In [1]:
import numpy as np

class Multilayer2DCA:
    def __init__(self, data):
        self.__parse(data)
        self.__size_x = 100
        self.__size_y = 100
        self.grid = np.zeros((3, 100, 100), dtype=int)

    def evolve(self):
        self.__new_grid = np.zeros((self.num_layers, self.size_x, self.size_y), dtype=int)

        self.__evolve_humidity() #Evolucionem capa d'humitat
        self.__evolve_fuel() #Evolucionem capa de combustible
        self.__evolve_fire()  #Evolucionem capa de foc

        self.grid = self.__new_grid

    def get_neighbors(self, layer, i, j):
        neighbors = []
        for x in range(-1, 2):
            for y in range(-1, 2):
                if x == 0 and y == 0:
                    continue
                ni = (i + x) if (i+x < self.__size_x) and (i+x >=0) else None # Assumpció de frontera tancada
                nj = (j + y) if (j+y < self.__size_y) and (j+y >=0) else None
                if ni and nj: neighbors.append(self.grid[layer, ni, nj]) #Només afegeix si són vàlids

        return neighbors
    
    def __parse(file):
        pass

    def __evolve_humidity(self):
        for i in range(self.__size_x):
                for j in range(self.__size_y):
                    neighbors_fire = self.get_neighbors(2, i, j)
                    current_humidity = self.grid[0,i,j]
                    self.__new_grid[0, i, j] = self.__humidity_func(neighbors_fire,current_humidity)

    def __evolve_fuel(self):
        for i in range(self.__size_x):
                for j in range(self.__size_y):
                    current_fuel = self.grid[1,i,j]
                    current_fire = self.grid[2,i,j]
                    self.__new_grid[1, i, j] = self.__fuel_func(current_fuel,current_fire)
    
    def __evolve_fire(self):
        for i in range(self.__size_x):
                for j in range(self.__size_y):
                    neighbors_fire = self.get_neighbors(2, i, j)
                    current_fuel = self.grid[1,i,j]
                    current_humidity = self.grid[0,i,j]
                    current_fire = self.grid[2,i,j]
                    self.__new_grid[2, i, j] = self.__fire_func(current_humidity,current_fuel,current_fire,neighbors_fire)    
        
    def simulate(self,steps):
        evolution_steps = [self.grid.copy()]

        for _ in range(steps):
             self.evolve()
             evolution_steps.append(self.grid.copy())

        return evolution_steps
    def __fire_func(current_humidity,current_fuel,current_fire,neighbors_fire):
        #-1: ja cremat 0: per cremar 1: cremant

        num_neighbours_fire = sum(1 for num in neighbors_fire if neighbors_fire == 1) #Nombre de veins que estàn cremant actualment
        if current_fire == -1: return -1 #Si ja estava cremat, no pot tornar a cremar mai
        if current_fire == 1:
            if current_fuel == 1: return -1 #Si al pas anterior quedava 1 de fuel, ara s'haurà acabat
            if current_fuel >1: return 1 # Si encara en queda, segueix cremant
        if current_fire == 0:
             if current_humidity == 0 and current_fuel >= 0 and num_neighbours_fire >0: # Si hi ha un vei cremat, la humitat es 0 i hi ha combustible,passa a cremar-se
                  return 1
             else:
                  return 0
             

    def __fuel_func(self,current_fuel,current_fire):
        res = current_fuel - 1 if current_fire == 1 else current_fuel # Si està cremat, anar cremant un a un
        return res if res >= 0 else 0


    def __humidity_func(self,neighbors,current_humidity):
        num_neighbours_fire = sum(1 for num in neighbors if neighbors == 1) #Nombre de veins que estàn cremant actualment.
        reduce_t = (1.5* np.log(num_neighbours_fire))+1 if num_neighbours_fire >0 else 0 #Reducció d'humitat degut a la presència de foc a caselles adjacents.
        res = current_humidity - reduce_t #Resultat del següent temps de la capa
        return res if res >= 0 else 0


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def visualize(evolution_steps):
    fig = plt.figure()
    ims = []
    for step in evolution_steps:
        im = plt.imshow(step, cmap='binary', animated=True)
        ims.append([im])
    ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True, repeat_delay=1000)
    plt.show()



In [ ]:
ca = Multilayer2DCA(data = "hola")

visualize(ca.simulate(steps = 100))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

class CellularAutomaton2D:
    def __init__(self, size_x, size_y, num_layers, rule_functions):
        self.size_x = size_x
        self.size_y = size_y
        self.num_layers = num_layers
        self.grid = np.zeros((num_layers, size_x, size_y), dtype=int)
        self.rule_functions = rule_functions

    def random_initial_state(self, density=0.5):
        self.grid[0] = np.random.choice([0, 1], size=(self.size_x, self.size_y), p=[1 - density, density])

    def evolve(self):
        new_grid = np.zeros((self.num_layers, self.size_x, self.size_y), dtype=int)
        for layer in range(self.num_layers):
            for i in range(self.size_x):
                for j in range(self.size_y):
                    neighbors = self.get_neighbors(layer, i, j)
                    new_grid[layer, i, j] = self.rule_functions[layer](self.grid[layer, i, j], neighbors)
        self.grid = new_grid

    def get_neighbors(self, layer, i, j):
        neighbors = []
        for x in range(-1, 2):
            for y in range(-1, 2):
                if x == 0 and y == 0:
                    continue
                ni = (i + x) % self.size_x
                nj = (j + y) % self.size_y
                neighbors.append(self.grid[layer, ni, nj])
        return neighbors

def visualize(evolution_steps):
    fig = plt.figure()
    ims = []
    for step in evolution_steps:
        im = plt.imshow(step, cmap='binary', animated=True)
        ims.append([im])
    ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True, repeat_delay=1000)
    plt.show()

# Definir funciones de regla para cada capa
def rule_layer_1(current_state, neighbors):
    # Ejemplo de función de regla: Conway's Game of Life
    alive_neighbors = sum(neighbors)
    if current_state == 1:
        if alive_neighbors < 2 or alive_neighbors > 3:
            return 0
        else:
            return 1
    else:
        if alive_neighbors == 3:
            return 1
        else:
            return 0

def rule_layer_2(current_state, neighbors):
    # Ejemplo de función de regla: Regla de la mayoría
    alive_neighbors = sum(neighbors)
    if alive_neighbors >= 5:
        return 1
    else:
        return 0

# Tamaño del autómata y número de pasos de evolución
size_x = 50
size_y = 50
num_layers = 2
steps = 100

# Crear el autómata celular
rule_functions = [rule_layer_1, rule_layer_2]
ca = CellularAutomaton2D(size_x, size_y, num_layers, rule_functions)
ca.random_initial_state()

# Evolucionar el autómata y almacenar los pasos
evolution_steps = [ca.grid.copy()]
for _ in range(steps):
    ca.evolve()
    evolution_steps.append(ca.grid.copy())

# Visualizar la evolución del autómata
visualize(evolution_steps)